This is the initial notebook to explore the data and run the first models. 

In [19]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

from fastai.imports import *
from fastai.tabular.all import *

from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold

import eli5
from eli5.sklearn import PermutationImportance

from scipy.stats import skew, norm
from catboost import CatBoostRegressor
from feature_engine.transformation import YeoJohnsonTransformer

import optuna
from optuna.samplers import TPESampler

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

Based on initial analysis the train and test datasets have similar characteristics, so it will be easier to combine them for imputation and data analysis work. 

In [3]:
train_test = pd.concat([train, test], ignore_index=True)

In [4]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

It looks like Alley, FireplaceQu, PoolQC, Fence and MiscFeature have significant numbers of missing data. So it will be best to eliminate those rows. There are a number of rows that have less than 5 rows with missing data. Since some of these are categorical and some are continuous data, their missing data will be replaced with the most frequent value. 

In [5]:
drop_high_nan=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_test=train_test.drop(drop_high_nan,axis=1)
small_nan_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageCars', 
                  'GarageArea','SaleType', 'SaleCondition']
small_impute = SimpleImputer(strategy='most_frequent')
train_test[small_nan_cols] = pd.DataFrame(small_impute.fit_transform(train_test[small_nan_cols]),columns=small_nan_cols)

The following columns seem to have one value significantly larger than the rest, and it would probably be best to use the mode, or most common, value to feel each NaN value: MasVnrType, MasVnrArea, BsmtCond, BsmtExposure, BsmtFinType2, GarageType, GarageFinish, GarageQual, and GarageCond. That is represents 9 out of the 13 columns. 

BsmtQual has two values larger than the rest: Gd and TA. But it only has 2.8% NaNs, so simply using the mode might be good enough.  

GarageYrBlt has 59 NaNs out 2919 rows which is only 2%. It has a dispersed set of values, so it might be easiest just to have any NaNs have the same value as YearBuilt. 

BsmtFinType1 has only 2.7% value of NaNs, and most two of its largest values are GLQ and Unf. It might be easiest to use the mode here. 

LotFrontage has 486 NaNs out of 2919 rows which is a pretty high 16.7%. It has a dispersed range of values, but looking at its characteristics from the describe function above, it seems to have a pretty even distribution with a mean of 10,168 and a median of 9,453. So using the mean to fill in the NaNs seems reasonable. If it turns out this value has a high impact on the predictability of the SalePrice, then it might be good to revisit this assumption. 

In [6]:
mode_cols = ['MasVnrType', 'MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'GarageType', 'GarageFinish', 
             'GarageQual','GarageCond', 'BsmtQual', 'BsmtFinType1']
mode_impute = SimpleImputer(strategy='most_frequent')
train_test[mode_cols] = pd.DataFrame(mode_impute.fit_transform(train_test[mode_cols]),columns=mode_cols)
train_test['LotFrontage'].fillna((train_test['LotFrontage'].mean()), inplace=True)
train_test['GarageYrBlt'] = train_test['GarageYrBlt'].fillna(train_test['YearBuilt'])

In [6]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

LotFrontage      486
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageQual       159
GarageCond       159
SalePrice       1459
dtype: int64

In [7]:
train_test['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', nan, 'Fa'], dtype=object)

In [8]:
train_test['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', nan, 'LwQ'], dtype=object)

In [9]:
train_test.BsmtQual = train_test.BsmtQual.replace({"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0})

In [10]:
train_test.BsmtFinType1 = train_test.BsmtFinType1.replace({"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1,
                                                         "NA": 0})

It will be necessary to identify all the columns that have non-numeric object values and then convert them to numeric values. 

In [11]:
obj_cols = list(train_test.select_dtypes(['object']).columns)
obj_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [12]:
for column in obj_cols:
     train_test[column] = pd.factorize(train_test[column], sort=True)[0]

C:\Users\tompe\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


To create some new columns that might compound the effects of the existing columns with higher impact on the xgb score. 

In [13]:
train_test['QualCondSum'] = train_test['OverallQual'] + train_test['OverallCond']
train_test['RemodTime'] = train_test['YearRemodAdd'] - train_test['YearBuilt']
train_test['BsmtFinTypeSF1'] = train_test['BsmtFinType1'] * train_test['BsmtFinSF1']
train_test['TotalFlrSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF']
train_test['TotalFinSF'] = train_test['GrLivArea'] + train_test['BsmtFinSF1']
train_test['GarageCarArea'] = train_test['GarageArea'] * train_test['GarageCars']
train_test['TotalSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF'] + train_test['TotalBsmtSF']

To now eliminate the columns that have negative or zero influence on the xgb score. 

In [14]:
train_test.drop(['Utilities','Street','TotRmsAbvGrd','3SsnPorch','YrSold','Exterior2nd','LotConfig',
                              'HouseStyle','EnclosedPorch','WoodDeckSF','Foundation','RoofMatl','Electrical',
                              'GarageType','LotFrontage','SaleType','MoSold','BsmtExposure','1stFlrSF',
                              'BsmtFinSF1','Exterior1st','KitchenQual','TotalFlrSF'], axis=1, inplace=True)

To look at how many features have a skew above 0.6, since high skew can be an issue in regression analysis. 

In [15]:
skew_features = train_test.apply(lambda x: skew(x)).sort_values(ascending=False)

In [16]:
high_skew = skew_features[skew_features > 0.6]
skew_index = high_skew.index

In [17]:
skew_index

Index(['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', 'Heating',
       'Condition2', 'LandSlope', 'KitchenAbvGr', 'ScreenPorch',
       'BsmtHalfBath', 'BsmtFinSF2', 'Condition1', 'OpenPorchSF', 'BldgType',
       'RemodTime', 'RoofStyle', 'MSSubClass', 'GarageCarArea', 'GrLivArea',
       'TotalFinSF', 'TotalSF', '2ndFlrSF', 'Fireplaces', 'HalfBath',
       'BsmtFullBath'],
      dtype='object')

In [18]:
print("There are {} numerical features with Skew > 0.6 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skewness

There are 25 numerical features with Skew > 0.6 :


,Skew
MiscVal,21.947195
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
Heating,12.078788
Condition2,12.060093
LandSlope,4.975157
KitchenAbvGr,4.302254
ScreenPorch,3.946694
BsmtHalfBath,3.931594


To normalize the features with skew above 0.6.

In [20]:
yjt = YeoJohnsonTransformer()
yjt.fit(train_test[skew_index])
train_test[skew_index] = yjt.transform(train_test[skew_index])

In [22]:
skew_features = train_test.apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 0.6]
skew_index = high_skew.index
skew_index

Index(['PoolArea', 'LowQualFinSF', 'MiscVal', 'LandSlope', 'BsmtHalfBath',
       'ScreenPorch', 'BsmtFinSF2', 'BldgType', 'Condition2'],
      dtype='object')

In [23]:
print("There are {} numerical features with Skew > 0.6 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skewness

There are 9 numerical features with Skew > 0.6 :


,Skew
PoolArea,14.884318
LowQualFinSF,8.365937
MiscVal,5.037498
LandSlope,4.213417
BsmtHalfBath,3.707260
ScreenPorch,2.915215
BsmtFinSF2,2.356620
BldgType,1.765554
Condition2,1.384128


In [20]:
#for i in skew_index:
#    train_test[i] = np.log1p(train_test[i])

C:\Users\tompe\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


To create a column with the log of the SalePrice to match the evaluation process in the contest. 

In [24]:
train_test['LogSalePrice'] = train_test['SalePrice'].apply(np.log)

To separate the train_test dataset back into the train and test datasets, identify the independent and dependent columns, and create the validation split.

In [25]:
train = train_test[train_test['SalePrice'].notnull()].copy()
test = train_test[train_test['SalePrice'].isnull()].drop(['SalePrice','LogSalePrice'],axis=1)
X = train.drop(['SalePrice','LogSalePrice'],axis=1)
y = train.LogSalePrice

In [23]:
'''def xgb_objective(trial):
    
    param = {  
        #'tree_method':method,
        #'objective': 'reg:squaredlogerror',
        'eval_metric': 'rmse',
        'lambda': trial.suggest_float('lambda', 0, 10.0),
        'alpha': trial.suggest_float('alpha', 0, 10.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1,1.0),
        'subsample': trial.suggest_float('subsample', 0.2,1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001,0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100,3000),
        'max_depth': trial.suggest_categorical('max_depth', [2,3,4,5,6,7,8,9,10]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'num_parallel_tree': trial.suggest_int('num_parallel_tree',1,1),
        }

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse'''

In [24]:
'''study_xgb = optuna.create_study(sampler=TPESampler(), direction='minimize')
study_xgb.optimize(xgb_objective, n_trials=1000)
print('Number of finished trials:', len(study_xgb.trials))
print('Best trial:', study_xgb.best_trial.params)'''

[I 2023-06-06 07:55:24,660] A new study created in memory with name: no-name-8e48ddf0-2766-4099-bc18-c57b6583c40c
[I 2023-06-06 07:55:27,892] Trial 0 finished with value: 0.15449259040773541 and parameters: {'lambda': 5.005512412442891, 'alpha': 7.507617123368103, 'colsample_bytree': 0.5172618900102177, 'subsample': 0.28895669552474684, 'learning_rate': 0.06955157564922773, 'n_estimators': 2231, 'max_depth': 9, 'min_child_weight': 6, 'num_parallel_tree': 1}. Best is trial 0 with value: 0.15449259040773541.
[I 2023-06-06 07:55:31,762] Trial 1 finished with value: 0.13641266132782368 and parameters: {'lambda': 9.712140075283925, 'alpha': 6.987322929271834, 'colsample_bytree': 0.9948400112674631, 'subsample': 0.9455138711490416, 'learning_rate': 0.01663248887679928, 'n_estimators': 2453, 'max_depth': 3, 'min_child_weight': 4, 'num_parallel_tree': 1}. Best is trial 1 with value: 0.13641266132782368.
[I 2023-06-06 07:55:34,273] Trial 2 finished with value: 0.15202119138544057 and parameters

[I 2023-06-06 07:56:18,066] Trial 20 finished with value: 0.12514377783784233 and parameters: {'lambda': 2.180374322134986, 'alpha': 2.0955928782841804, 'colsample_bytree': 0.2047772530002056, 'subsample': 0.5610717025710661, 'learning_rate': 0.023349243814958648, 'n_estimators': 1541, 'max_depth': 2, 'min_child_weight': 6, 'num_parallel_tree': 1}. Best is trial 19 with value: 0.11033085615589797.
[I 2023-06-06 07:56:21,053] Trial 21 finished with value: 0.1090806721241984 and parameters: {'lambda': 1.2873257231394375, 'alpha': 1.0302040674616142, 'colsample_bytree': 0.444424840534136, 'subsample': 0.5632870298355229, 'learning_rate': 0.025907671991715484, 'n_estimators': 1123, 'max_depth': 6, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 21 with value: 0.1090806721241984.
[I 2023-06-06 07:56:23,683] Trial 22 finished with value: 0.14973650991920073 and parameters: {'lambda': 1.1375735776627849, 'alpha': 1.3042580152021617, 'colsample_bytree': 0.42681287965974557, 'subs

[I 2023-06-06 07:57:29,846] Trial 40 finished with value: 0.1525342887620296 and parameters: {'lambda': 4.677566585686249, 'alpha': 0.7569829006274198, 'colsample_bytree': 0.4855644178502802, 'subsample': 0.716862543107563, 'learning_rate': 0.03706059578951099, 'n_estimators': 370, 'max_depth': 3, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 21 with value: 0.1090806721241984.
[I 2023-06-06 07:57:34,191] Trial 41 finished with value: 0.1353370494923925 and parameters: {'lambda': 2.909699801648847, 'alpha': 2.0296881704211196, 'colsample_bytree': 0.38398860438615784, 'subsample': 0.5295630710273964, 'learning_rate': 0.0382498805531267, 'n_estimators': 1299, 'max_depth': 6, 'min_child_weight': 6, 'num_parallel_tree': 1}. Best is trial 21 with value: 0.1090806721241984.
[I 2023-06-06 07:57:39,268] Trial 42 finished with value: 0.11411518796798986 and parameters: {'lambda': 2.260384467778971, 'alpha': 1.7430613247745856, 'colsample_bytree': 0.38743059856151657, 'subsample':

[I 2023-06-06 07:59:35,124] Trial 60 finished with value: 0.11358476826674026 and parameters: {'lambda': 1.435115948690739, 'alpha': 3.7587311553239195, 'colsample_bytree': 0.37060369425378464, 'subsample': 0.5800483024681101, 'learning_rate': 0.01809680041270406, 'n_estimators': 2374, 'max_depth': 2, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 44 with value: 0.10740699373958382.
[I 2023-06-06 07:59:41,099] Trial 61 finished with value: 0.13367274688921862 and parameters: {'lambda': 1.4275710393178218, 'alpha': 3.258011664012228, 'colsample_bytree': 0.36536722347656975, 'subsample': 0.5749007962728189, 'learning_rate': 0.018873688894383508, 'n_estimators': 2341, 'max_depth': 2, 'min_child_weight': 7, 'num_parallel_tree': 1}. Best is trial 44 with value: 0.10740699373958382.
[I 2023-06-06 07:59:46,404] Trial 62 finished with value: 0.14407559245147164 and parameters: {'lambda': 2.028696809275548, 'alpha': 3.7906406046435586, 'colsample_bytree': 0.403982528487307, 'subs

[I 2023-06-06 08:01:10,998] Trial 80 finished with value: 0.14248380659608137 and parameters: {'lambda': 3.1724023780981785, 'alpha': 2.6370522951517406, 'colsample_bytree': 0.4871320344324041, 'subsample': 0.46202624117017144, 'learning_rate': 0.028498599482763183, 'n_estimators': 829, 'max_depth': 6, 'min_child_weight': 6, 'num_parallel_tree': 1}. Best is trial 44 with value: 0.10740699373958382.
[I 2023-06-06 08:01:16,074] Trial 81 finished with value: 0.12674532397045568 and parameters: {'lambda': 2.193727951423217, 'alpha': 1.7068752487716092, 'colsample_bytree': 0.38740186511577523, 'subsample': 0.5471548737486193, 'learning_rate': 0.043873922436891984, 'n_estimators': 1709, 'max_depth': 6, 'min_child_weight': 6, 'num_parallel_tree': 1}. Best is trial 44 with value: 0.10740699373958382.
[I 2023-06-06 08:01:21,045] Trial 82 finished with value: 0.13387186400956244 and parameters: {'lambda': 3.7466592850481506, 'alpha': 2.1765597504926513, 'colsample_bytree': 0.43093752209245223, '

...

[I 2023-06-06 09:05:22,464] Trial 920 finished with value: 0.11951212343413994 and parameters: {'lambda': 1.1454992955868548, 'alpha': 0.6551125031619827, 'colsample_bytree': 0.9459801253329516, 'subsample': 0.34149844484134756, 'learning_rate': 0.08416169037591989, 'n_estimators': 1018, 'max_depth': 6, 'min_child_weight': 5, 'num_parallel_tree': 1}. Best is trial 173 with value: 0.09909467168372289.
[I 2023-06-06 09:05:28,290] Trial 921 finished with value: 0.11668940378741083 and parameters: {'lambda': 9.47253281624242, 'alpha': 1.0068867777481922, 'colsample_bytree': 0.458919205147025, 'subsample': 0.38648342789123113, 'learning_rate': 0.03183808731295597, 'n_estimators': 2152, 'max_depth': 6, 'min_child_weight': 10, 'num_parallel_tree': 1}. Best is trial 173 with value: 0.09909467168372289.
[I 2023-06-06 09:05:33,224] Trial 922 finished with value: 0.13728937813723052 and parameters: {'lambda': 3.282746851128523, 'alpha': 2.204496909340925, 'colsample_bytree': 0.7602481606777884, '

[I 2023-06-06 09:06:41,198] Trial 940 finished with value: 0.1260186890880539 and parameters: {'lambda': 1.4134276373290333, 'alpha': 1.5572406384633097, 'colsample_bytree': 0.2613760203126053, 'subsample': 0.5630882354300253, 'learning_rate': 0.07509516955420019, 'n_estimators': 1888, 'max_depth': 6, 'min_child_weight': 5, 'num_parallel_tree': 1}. Best is trial 173 with value: 0.09909467168372289.
[I 2023-06-06 09:06:42,001] Trial 941 finished with value: 0.14340988489761958 and parameters: {'lambda': 3.4052382676833446, 'alpha': 5.199467358436928, 'colsample_bytree': 0.30112650968135674, 'subsample': 0.5852550079066619, 'learning_rate': 0.05135512543881216, 'n_estimators': 183, 'max_depth': 10, 'min_child_weight': 1, 'num_parallel_tree': 1}. Best is trial 173 with value: 0.09909467168372289.
[I 2023-06-06 09:06:46,771] Trial 942 finished with value: 0.12891210356617733 and parameters: {'lambda': 2.824051199299805, 'alpha': 2.520485247262162, 'colsample_bytree': 0.8666836823260651, 's

[I 2023-06-06 09:07:59,426] Trial 960 finished with value: 0.12595104249373462 and parameters: {'lambda': 5.105348135167411, 'alpha': 9.187319215140963, 'colsample_bytree': 0.3160875829109166, 'subsample': 0.7401945151694266, 'learning_rate': 0.06746443479535177, 'n_estimators': 2116, 'max_depth': 6, 'min_child_weight': 10, 'num_parallel_tree': 1}. Best is trial 173 with value: 0.09909467168372289.
[I 2023-06-06 09:08:04,687] Trial 961 finished with value: 0.12747634367857952 and parameters: {'lambda': 2.6529286444748874, 'alpha': 0.9577665105316276, 'colsample_bytree': 0.238036900447376, 'subsample': 0.8055874870530412, 'learning_rate': 0.08598386417015848, 'n_estimators': 2048, 'max_depth': 5, 'min_child_weight': 5, 'num_parallel_tree': 1}. Best is trial 173 with value: 0.09909467168372289.
[I 2023-06-06 09:08:08,859] Trial 962 finished with value: 0.11654195782895242 and parameters: {'lambda': 4.063458752641315, 'alpha': 1.803974473278013, 'colsample_bytree': 0.3782130016914927, 'su

[I 2023-06-06 09:09:21,759] Trial 980 finished with value: 0.14296306042259949 and parameters: {'lambda': 2.0106902365070383, 'alpha': 1.1414471817085006, 'colsample_bytree': 0.5529730980960956, 'subsample': 0.8587219834766833, 'learning_rate': 0.0333226847498769, 'n_estimators': 1058, 'max_depth': 3, 'min_child_weight': 3, 'num_parallel_tree': 1}. Best is trial 173 with value: 0.09909467168372289.
[I 2023-06-06 09:09:26,342] Trial 981 finished with value: 0.1299011692135734 and parameters: {'lambda': 2.970463134224524, 'alpha': 2.6133868281943626, 'colsample_bytree': 0.36993416764433146, 'subsample': 0.7768779889606006, 'learning_rate': 0.028394779129190306, 'n_estimators': 1695, 'max_depth': 6, 'min_child_weight': 4, 'num_parallel_tree': 1}. Best is trial 173 with value: 0.09909467168372289.
[I 2023-06-06 09:09:27,918] Trial 982 finished with value: 0.1277912448878613 and parameters: {'lambda': 4.8012841808924165, 'alpha': 0.7080502884954265, 'colsample_bytree': 0.2613811843823692, '

Number of finished trials: 1000
Best trial: {'lambda': 1.32754289013568, 'alpha': 1.580513088436275, 'colsample_bytree': 0.34112979113700337, 'subsample': 0.49829211790557504, 'learning_rate': 0.04245317476432714, 'n_estimators': 1145, 'max_depth': 6, 'min_child_weight': 6, 'num_parallel_tree': 1}


In [26]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [27]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=6))
    return rmse

In [28]:
params_xgb= {'lambda': 1.32754289013568, 
             'alpha': 1.580513088436275, 
             'colsample_bytree': 0.34112979113700337, 
             'subsample': 0.49829211790557504, 
             'learning_rate': 0.04245317476432714, 
             'n_estimators': 1145,
             'max_depth': 6, 
             'min_child_weight': 6, 
             'num_parallel_tree': 1}

In [29]:
xgb_model = xgb.XGBRegressor(**params_xgb)

In [30]:
print(rmse_cv(xgb_model,X,y).mean())

0.12540821139707464


In [31]:
submit = test[['Id']]
submit = submit.reset_index(drop=True)

In [32]:
xgb_model.fit(X,y)

XGBRegressor(alpha=1.580513088436275, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.34112979113700337, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=1.32754289013568,
             learning_rate=0.04245317476432714, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             n_estimators=1145, n_jobs=None, num_parallel_tree=1, ...)

In [33]:
submit_predict = xgb_model.predict(test)
submit_predict = np.exp(submit_predict)

In [34]:
submit['SalePrice'] = submit_predict

In [35]:
submit.to_csv('submit_xgb_optuna.csv', index=False)

Reducing feature skewness actually slightly increased the Kaggle score. 